In [1]:
import numpy as np
import cv2
from sklearn import datasets
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def detect_lines(img):
#     plt.imshow(img)
    lines_to_return = []
    # detekcija koordinata linije koristeci Hough transformaciju
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges_img = cv2.Canny(gray_img, 50, 150, apertureSize=3)
#     plt.imshow(edges_img, "gray")
    
    # minimalna duzina linije
    min_line_length = 400
    
    # Hough transformacija
    lines = cv2.HoughLinesP(image=edges_img, rho=1, theta=np.pi/180, threshold=10, lines=np.array([]),
                            minLineLength=min_line_length, maxLineGap=20)
    
#     print("Detektovane linije [[x1 y1 x2 y2]]: \n", lines)
#     print(len(lines))
    
    for line in lines:
        x1 = line[0][0]
        y1 = line[0][1]
        x2 = line[0][2]
        y2 = line[0][3]
        lines_to_return.append([x1, y1, x2, y2])
        
    return lines_to_return

In [3]:
def process_video(video_path):
    # procesiranje jednog videa
    # priprema pomocnih promenljivih
    minline = {}
    maxline = {}
    
    number_of_hits = 0
    hits_left = 0
    hits_rigth = 0
    # ucitavanje videa
    frame_num = 0
    cap = cv2.VideoCapture(video_path)
    cap.set(1, frame_num) # indeksiranje frejmova
    
    # analiza videa frejm po frejm
    while True:
        frame_num += 1
        ret_val, frame = cap.read()
        # ako frejm nije zahvacen
        if not ret_val:
            print(video_path)
            print(number_of_hits)
            print(hits_left)
            print(hits_rigth)
            break
        
        if frame_num == 1: # ako je prvi frejm, detektuj liniju
            lines = detect_lines(frame)
            minline = lines[0]
            maxline = lines[0]
            for line in lines:
                if line[2] - line[0] == 0:
#                     print("Linija: ", line)
                    if(line[0] > maxline[0]):
                        maxline = line
                    if(line[0] < minline[0]):
                        minline = line
                    
#                     cv2.line(frame, (line[0], line[1]), (line[2], line[3]), (0, 0, 255), 3, cv2.LINE_AA)

            cv2.line(frame, (minline[0], minline[1]), (minline[2], minline[3]), (0, 0, 255), 3, cv2.LINE_AA)
            cv2.line(frame, (maxline[0], maxline[1]), (maxline[2], maxline[3]), (0, 0, 255), 3, cv2.LINE_AA)
            plt.imshow(frame)
            print(minline)
            print(maxline)
        
        else:
            ############
            img = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY) # konvert u grayscale
            plt.imshow(img, 'gray')
            
            ret, image_barcode_bin = cv2.threshold(img, 80, 255, cv2.THRESH_BINARY)
            plt.imshow(img, 'gray')
            
            contours, hierarchy = cv2.findContours(image_barcode_bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
            img = frame.copy()
            cv2.drawContours(img, contours, -1, (0, 255, 0), 1)
            plt.imshow(img)
            
            contours_circle = [] #ovde ce biti samo konture koje pripadaju bar-kodu
            centers_circle = []
            for contour in contours: # za svaku konturu
                ((x, y), radius) = cv2.minEnclosingCircle(contour)
                if radius > 3.6 and radius < 5: # uslov da kontura pripada bar-kodu
                    contours_circle.append(contour) # ova kontura pripada bar-kodu
                    centers_circle.append([x, y])
#                     print(x)
                    if abs(minline[0] - x) < 16 or abs(maxline[0] - x) < 16:
                        number_of_hits += 1
#                         print("HIT")
                    if abs(minline[0] - x) < 16:
                        hits_left += 1
#                         print(x)
#                         print("LEFT HIT")
                    if abs(maxline[0] - x) < 16:
                        hits_rigth += 1
#                         print(x)
#                         print("RIGTH HIT")
                    
#             img = frame.copy()
#             cv2.drawContours(img, contours_circle, -1, (255, 0, 0), 10)
#             print(len(contours_circle))
#             print(centers_circle)
#             plt.imshow(img)
#             return


In [ ]:
process_video("data/video1.mp4")
process_video("data/video2.mp4")
process_video("data/video3.mp4")
process_video("data/video4.mp4")
process_video("data/video5.mp4")
process_video("data/video6.mp4")
process_video("data/video7.mp4")
process_video("data/video8.mp4")
process_video("data/video9.mp4")
process_video("data/video10.mp4")

[288, 580, 288, 108]
[671, 508, 671, 108]
data/video1.mp4
7
2
5
[288, 580, 288, 108]
[672, 501, 672, 68]
data/video2.mp4
4
3
1
[288, 580, 288, 108]
[671, 505, 671, 71]
data/video3.mp4
11
5
6
[288, 580, 288, 108]
[671, 505, 671, 71]
